# [Day 11](https://adventofcode.com/2020/day/11): Seating System

In [1]:
import numpy as np

with open("../data/11.txt", "r") as f:
    lines = [l.strip() for l in f.readlines()]

seatingplan = np.array(list(map(list, lines)))

## Part 1

In [2]:
from itertools import product

shifts = np.array(list(product((-1, 0, 1), repeat=2)))

def occupy(seats):
    def density(occupied):
        nbhs = (np.roll(occupied, s, axis=(0, 1)) for s in shifts)
        return np.sum(tuple(nbhs), axis=0)
    def move(occupied):
        d = density(occupied)
        return (seats & (d == 0)) | (occupied & (d <= 4))
    return move

def fixedpoint(f, x):
    while ((fx := f(x)) != x).any():
        x = fx
    return x

seats = np.pad(seatingplan == "L", 1)
assert np.sum(fixedpoint(occupy(seats), seats)) == 2344

## Part 2

In [3]:
import numpy.ma as ma
from itertools import accumulate

UNOCCUPIED = -np.inf

def ray(x):
    r = accumulate(x, reluadd, initial=0)
    r = np.array(list(r)[1:], dtype=bool)
    r = r & shift(r)
    return r | shift(r) | shift(x == 1)

def reluadd(x, y):
    return np.maximum(0, x + y)

def shift(x):
    return np.pad(x[:-1], ((1, 0), (0, 0)))

def rayrev(x):
    return ray(x[::-1])[::-1]

def illumdiag(diags, illum):
    def illuminate(x):
        d = diags(x)
        r = illum(d.data)
        return r[~d.mask].reshape(x.shape)
    return illuminate

def diagonalizer(shear):
    def diagonals(x):
        m, n = x.shape
        i, j = np.indices(x.shape)
        j = j + shear(m)[:, np.newaxis]
        diag = np.full((m, m + n - 1), UNOCCUPIED)
        mask = np.ones(diag.shape, dtype=bool)
        diag[i, j] = x
        mask[i, j] = False
        return ma.array(diag, mask=mask)
    return diagonals

diags = diagonalizer(lambda m: np.arange(m - 1, -1, -1))
antidiags = diagonalizer(np.arange)

illuminate = (
    ray,                           # S
    rayrev,                        # N
    lambda x: ray(x.T).T,          # E
    lambda x: rayrev(x.T).T,       # W
    illumdiag(diags, ray),         # SE
    illumdiag(diags, rayrev),      # NW
    illumdiag(antidiags, ray),     # SW
    illumdiag(antidiags, rayrev))  # NE

def occupy2(seats):
    def visible(occupied):
        occ = occupied.astype(float)
        occ[seats & ~occupied] = UNOCCUPIED
        lineofsight = (illum(occ) for illum in illuminate)
        return np.sum(tuple(lineofsight), axis=0)
    def move(occupied):
        v = occupied + visible(occupied)
        return (seats & (v == 0)) | (occupied & (v <= 5))
    return move

seats2 = seats[1:-1, 1:-1]
assert 2076 == np.sum(fixedpoint(occupy2(seats2), seats2))